In [5]:
import tempfile
import os

import tensorflow as tf

from tensorflow import keras
from quanteyes.dataloader.dataset_tf import get_zipped_dataset

In [9]:
# Load MNIST dataset
# mnist = keras.datasets.mnist
# (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
base_path = "/mnt/sdb/data/Openedsdata2020/openEDS2020-GazePrediction-1bit-otsu"
train_dataset = get_zipped_dataset(f"{base_path}/train", train=True).shuffle(1000)
val_dataset = get_zipped_dataset(f"{base_path}/validation", train=False).shuffle(
	1000
)

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential(
	[
		keras.layers.Conv2D(
			16,
			(3, 3),
			activation="relu",
			input_shape=(200, 320, 1),
			padding="same",
		),
		keras.layers.MaxPooling2D((2, 2)),
		keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
		keras.layers.MaxPooling2D((2, 2)),
		keras.layers.Flatten(),
		keras.layers.Dense(64, activation="relu"),
		keras.layers.Dense(3),
	]
)

# Train the digit classification model
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss=tf.keras.losses.CosineSimilarity(
            axis=1, reduction=tf.keras.losses.Reduction.SUM
        ),
        metrics=["mse", "cosine_similarity"],
    )

1000/1000 [==============================] - 58s 57ms/step - loss: -31.7921 - mse: 0.0699 - cosine_similarity: 0.9935 - val_loss: -31.1789 - val_mse: 0.2215 - val_cosine_similarity: 0.9743


In [ ]:
model.fit(
  train_dataset.batch(32).repeat(10),
  epochs=1,
  steps_per_epoch=1000,
  validation_data=val_dataset.batch(32).take(100),
)